In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow import keras

# Tensors and Operations

## constant

In [3]:
t=tf.constant([[1.,2.,3.],[4.,5.,6.]])  # matrix
t


<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [4]:
tf.constant(42)  # scaler

<tf.Tensor: shape=(), dtype=int32, numpy=42>

In [5]:
t.shape  #shape

TensorShape([2, 3])

In [6]:
t.dtype  # data type

tf.float32

In [7]:
# indexing
t[:,1:]

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[2., 3.],
       [5., 6.]], dtype=float32)>

In [8]:
t[...,1,tf.newaxis]

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[2.],
       [5.]], dtype=float32)>

In [9]:
t+10

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[11., 12., 13.],
       [14., 15., 16.]], dtype=float32)>

In [12]:
tf.square(t)  # square of array

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)>

In [ ]:
t @ tf.transpose(t)  # @ = matmultipaction

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[14., 32.],
       [32., 77.]], dtype=float32)>

#### Keras Low Level API

In [11]:
k = keras.backend
k.square(k.transpose(t)) +10

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[11., 26.],
       [14., 35.],
       [19., 46.]], dtype=float32)>

#### Tensor and NumPy


Notice that NumPy uses 64-bit precision by default, while Tensor‐
Flow uses 32-bit. This is because 32-bit precision is generally more
than enough for neural networks, plus it runs faster and uses less
RAM. So when you create a tensor from a NumPy array, make sure
to set dtype=tf.float32.

In [14]:
# Tensors and NumPy
a = np.array([2.,4.,5.])
tf.constant(a)

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([2., 4., 5.])>

In [15]:
tf.square(a)

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([ 4., 16., 25.])>

In [16]:
np.square(t)

array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)

#### Type conversions


In [24]:
# tf.constant(2.) + tf.constant(40) # gives error
# tf.constant(2.) + tf.constant(40.,dtype=tf.float64)  # it also gives the error 

tf.constant(2.) + tf.constant(40.) # default float32


<tf.Tensor: shape=(), dtype=float32, numpy=42.0>

## Variables

In [28]:
# variable
v = tf.Variable([[1.,2.,3.],[4.,5.,6.]])
v

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [29]:
v.assign(2*v)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2.,  4.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[ 2.,  4.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

In [36]:
v[0,1].assign(42)


<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2., 42.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

In [40]:
v[:,2].assign([1.,0])

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2., 42.,  1.],
       [ 8., 10.,  0.]], dtype=float32)>

In [42]:
v.scatter_nd_update(indices=[[0,0],[1,2]],updates=[100.,200.])

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[100.,  42.,   1.],
       [  8.,  10., 200.]], dtype=float32)>